In [101]:
import pymongo as pm
from pymongo import MongoClient
import pprint as pp
import re
import numpy as np

In [102]:
cluster = MongoClient("mongodb://localhost:27017")

In [103]:
print(cluster)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [104]:
db = cluster['Plagchecker_local']
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Plagchecker_local')


In [105]:
lc_col = db.get_collection('leetcode_solutions')

In [106]:
sub_col = db.get_collection('csi_submissions')

In [109]:
def winnowing_fingerprint(text, window_size=4):
    text = "".join(text.split())
    n = len(text)
    five_grams = []
    
    for i in range(n - 4):
        five_grams.append(text[i:i+5])
        
    hash_values = []
    for gram in five_grams:
        hash_values.append(sum(ord(char) for char in gram))

    windows = []

    for i in range(len(hash_values) - window_size + 1):
        window = hash_values[i:i + window_size]
        windows.append(window)

    fingerprints = [min(window) for window in windows]

    return fingerprints

def jaccard_similarity(doc1, doc2):
    # Convert fingerprints to sets
    set_doc1 = set(doc1)
    set_doc2 = set(doc2)
    
    # Calculate Jaccard similarity
    jaccard_sim = len(set_doc1.intersection(set_doc2)) / len(set_doc1.union(set_doc2))
    
    return jaccard_sim

def check_similarity(fp_doc1, fp_doc2, threshold=0.4):
    
    jaccard_sim = jaccard_similarity(fp_doc1, fp_doc2)
    perc_similar = jaccard_sim * 100
    return perc_similar

lang = ['^java$', '^python$', '^c$', '^c\\+\\+$']
question_numbers = [9, 69, 190, 141, 605, 290, 4, 11, 17, 38, 283, 70, 242, 206, 230, 54, 567, 51, 127, 875, 402]


curr_qn = 9

for curr_qn in question_numbers:
    for i in lang:
        curr_lang = i
        print(curr_lang)
    
        email_list = []
        code_list = []
        for sub in sub_col.find({'$and': [
                                        {'language': {'$regex': i, '$options': 'i'}},
                                        {'questionNumber': curr_qn}
                                        ]}):
            email = sub['email']
            email_list.append(email)
    
            code = sub['code']
            code_list.append(code)
    
        # print(email_list)
        # print(len(email_list))
        # print(code_list)
        # print(len(code_list))
        # print("\n\n")
    
        n = len(code_list)
        similarity_matrix = np.zeros((n, n))
    
        for i in range(n):
            for j in range(n):
                text1 = code_list[i]
                text2 = code_list[j]
    
                fp2 = winnowing_fingerprint(text2) # fingerprints of doc 2
                fp1 = winnowing_fingerprint(text1) # fingerprints of doc 1
    
                perc_sim = check_similarity(fp1, fp2) 
                
                similarity_matrix[i, j] = perc_sim
    
        print("Similarity Matrix for language: {} and Question ID: {} :".format(curr_lang,curr_qn))
        print(np.array2string(similarity_matrix, precision=2, separator=', ', suppress_small=True))
        print(n)
        print("\n")

^java$
Similarity Matrix for language: ^java$ and Question ID: 9 :
[[100.  ,  33.81,  44.53,  31.88,  23.64,  27.78,  30.65,  42.52,  27.78,
   37.36,  37.67,  23.64,  27.78,  26.43,  26.24,  42.52,  39.84,  44.53,
   28.99,  33.55],
 [ 33.81, 100.  ,  33.81,  37.1 ,  23.87,  27.41,  44.23,  33.07,  27.41,
   34.32,  34.04,  23.87,  27.41,  27.91,  34.96,  33.07,  35.77,  33.81,
   31.75,  37.32],
 [ 44.53,  33.81, 100.  ,  36.84,  22.89,  29.58,  33.88,  48.36,  29.58,
   35.03,  39.58,  22.89,  29.58,  28.26,  31.85,  48.36,  42.06, 100.  ,
   30.88,  30.19],
 [ 31.88,  37.1 ,  36.84, 100.  ,  21.29,  36.59,  36.45,  36.36,  36.59,
   31.95,  30.28,  21.29,  36.59,  49.07,  52.83,  36.36,  35.83,  36.84,
   54.29,  33.57],
 [ 23.64,  23.87,  22.89,  21.29, 100.  ,  22.58,  15.86,  23.03,  22.58,
   39.2 ,  18.97, 100.  ,  22.58,  20.39,  24.32,  23.03,  17.83,  22.89,
   25.17,  26.79],
 [ 27.78,  27.41,  29.58,  36.59,  22.58, 100.  ,  28.7 ,  28.46, 100.  ,
   27.84,  27.21,  22.58